In [1]:
from Data.BinancePriceFetcher import *
from Data.GetOrderBook import *
from Utils.config import *
from Strats.MeanReversionStrat import *
from Strats.RegressionStrat import *
from Strats.RSI import *

print("Trading parameters (edit in config.py file):")
print(f"Tickers: {tickers}")
print(f"Stop loss: {stoploss}")
print(f"Max drawdown duration: {drawdown_duration}")
print(f"Rolling window: {rolling}")
print(f"Portfolio Allocation method: {weight_method.__name__}")
print(f"Allow shorting in weight allocation: {short}")
print(f"Train start date: {start_date}")
print(f"Trading Frequency: {interval}")


symbol_manager = BinanceSymbolManager()
# Add symbols
for t in tickers:
    symbol_manager.add_symbol(t) # Success

passed_tickers = symbol_manager.get_symbols()
price_fetcher = BinancePriceFetcher(passed_tickers)

def get_signal(prices, portfolio_prices):
    signals = pd.DataFrame()

    mr_signals = {'mr': MeanReversionStrat(portfolio_prices[-MeanReversionStrat_PARAMS.lookback:].copy(), passed_tickers),
                  'rsi': RSI(portfolio_prices[-RegressionStrat_PARAMS.lookback_window-10:].copy(), passed_tickers)}

    trend_signals = {'lr': RegressionStrat(portfolio_prices[-RegressionStrat_PARAMS.lookback_window-10:].copy(), passed_tickers, 
                                           lookback_window=RegressionStrat_PARAMS.lookback_window, 
                                           regression_type=RegressionStrat_PARAMS.regression_type)}

    

    for ind, t in enumerate(passed_tickers):
        huber_result = hurst_exponent(portfolio_prices[t].values[-rolling:])
        if huber_result < Hurst_Type.mean_revert[-1]:
            # mean-revert
            mr_signal = mr_signals['mr'].generate_single_signal(
                t, prices[ind], lookback=MeanReversionStrat_PARAMS.lookback,
                execute_threshold=MeanReversionStrat_PARAMS.execute_threshold, 
                close_threshold=MeanReversionStrat_PARAMS.close_threshold)

            rsi_signal = mr_signals['rsi'].generate_single_signal(
                t, prices[ind], 
                rsi_period=RSI_PARAMS.rsi_period, stoch_period=RSI_PARAMS.stoch_period, 
                k_smooth=RSI_PARAMS.k_smooth, d_smooth=RSI_PARAMS.d_smooth)
            
            voted_signal = (mr_signal['signals'].item() + rsi_signal['signals'].item()) / len(mr_signals)
            voted_signal = 1 if voted_signal > 0.5 else 0

            voted_weights = (mr_signal['weights'].item() + rsi_signal['weights'].item()) / len(mr_signals)

            voted_exit_signal = (mr_signal['exit_signals'].item() + rsi_signal['exit_signals'].item()) / len(mr_signals)
            voted_exit_signal = 1 if voted_exit_signal > 0.5 else 0

            signal = mr_signal.copy()
            signal['signals'] = voted_signal
            signal['weights'] = voted_weights
            signal['exit_signals'] = voted_exit_signal
            
        else:
            # Regression or smth else
            signal = trend_signals['lr'].generate_single_signal(t, prices[ind], 
                                                     pca_components=RegressionStrat_PARAMS.pca_components, 
                                                     execute_threshold=RegressionStrat_PARAMS.execute_threshold, 
                                                     r2_exit=RegressionStrat_PARAMS.r2_exit)
        
        signals = pd.concat([signals, signal])
    signals['timestamp'] = datetime.now()
    
    signals = signals[['timestamp', 'Tickers', 'signals', 'weights', 'exit_signals', "Price"]]
    weights_sum = signals['weights'].sum()
    signals['weights'] = [i/weights_sum for i in signals['weights']]
    return signals

order_book_manager = OrderBookManager(passed_tickers)


Trading parameters (edit in config.py file):
Tickers: ['BTCUSDT', 'ETHUSDT', 'BNBUSDT']
Stop loss: 10000000.0
Max drawdown duration: 10000000.0
Rolling window: 1000
Portfolio Allocation method: hierarchical_risk_parity_weighting
Allow shorting in weight allocation: False
Train start date: 2025-04-14
Trading Frequency: 5m


In [2]:
# Fetch pair historical price up till latest point in time
portfolio_prices = price_fetcher.get_grp_historical_ohlcv(
        interval=interval,
        start_date=start_date,
        end_date=end_date
    )

portfolio_prices

,timestamp,BNBUSDT,BTCUSDT,ETHUSDT
0,2025-04-14 00:00:00,584.00,83774.19,1597.98
1,2025-04-14 00:05:00,584.44,83944.00,1603.76
2,2025-04-14 00:10:00,584.84,84146.00,1608.05
3,2025-04-14 00:15:00,585.50,84484.01,1611.30
4,2025-04-14 00:20:00,586.33,84760.99,1620.36
...,...,...,...,...
21227,2025-06-26 16:55:00,646.49,107196.05,2427.65
21228,2025-06-26 17:00:00,647.38,107399.00,2434.03
21229,2025-06-26 17:05:00,647.14,107353.65,2429.20
21230,2025-06-26 17:10:00,646.40,107220.59,2422.98


In [ ]:
# Main loop
while True:
    try:
        start_time = time.time()
        prices = []
        for t in tickers:
            # Print current state
            order_book = order_book_manager.fetch_order_book(t, limit = 5)
            prices += [[order_book.best_bid(), order_book.get_mid_price(), order_book.best_ask()]]
        signals = get_signal(prices, portfolio_prices)
        print(signals)
        
        #portfolio price scrape time start
        portfolio_prices_start = time.perf_counter()
        portfolio_prices = price_fetcher.get_grp_historical_ohlcv(
                interval=interval,
                start_date=start_date,
                end_date=now.strftime('%Y-%m-%d')
            )
        portfolio_prices_end = time.perf_counter()

        # Sleep to maintain ~frequency interval 
        time.sleep(interval_seconds - (portfolio_prices_end-portfolio_prices_start))

    except KeyboardInterrupt:
        logging.info("Stopping order book monitor...")
        break
    except Exception as e:
        logging.error(f"Error: {e}")
        time.sleep(5)  # Wait before retrying after error